# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import time
from collections import deque

import numpy as np
import torch

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import pandas as pd

init_notebook_mode(connected=True)

from unityagents import UnityEnvironment
env = UnityEnvironment(file_name="Tennis.app", no_graphics=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [3]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [4]:
from ddpg_agent import Agent
agent = Agent(num_agents=num_agents, state_size=state_size, action_size=action_size, random_seed=2)

Actor(
  (bn0): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=24, out_features=128, bias=True)
  (dropout1): Dropout(p=0.25)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (dropout2): Dropout(p=0.25)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)
Critic(
  (bn0): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fcs1): Linear(in_features=24, out_features=128, bias=True)
  (dropout1): Dropout(p=0.25)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=130, out_features=128, bias=True)
  (dropout2): Dropout(p=0.25)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in

In [5]:
scores = []                                                # global score
times = []

def train_ddpg(n_episodes=3000, print_every=100, num_agents=num_agents):
       
    scores_deque = deque(maxlen=100)                       # score's window of last 100 scores    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        agent.reset()                                      # reset the agent
        state = env_info.vector_observations               # get the current state
        score = np.zeros(num_agents)                       # reset score for an episod
        done = np.zeros(num_agents)
        t = 0
        while True:
            action = agent.act(state)                      # select an action       
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations      # get the next state
            reward = env_info.rewards                      # get the reward
            done = env_info.local_done                     # see if episode has finished
            agent.step(
                state, action, reward, next_state, done)   # take an action
            score += reward                                # update the score of the expisod
            state = next_state                             # roll over the state to next time step
            t += 1
            if np.any(done):
                break 
        times.append(t)
        scores_deque.append(np.mean(score))                # save agents' mean score to the window
        scores.append(np.mean(score))                      # save agents' scores to the global list
        
        text =\
            '\rEpisode {} | Score Average {:.3f}, perc 10, 50 , 90: , {:.2f}, {:.2f}, {:.2f} | Score: {:.2f} | Time {}'\
            .format(i_episode, np.mean(scores_deque), 
                    np.percentile(scores_deque, 10), np.percentile(scores_deque, 50), np.percentile(scores_deque, 90),
                    np.mean(score), t)
            
        if (i_episode % print_every == 0) or (np.mean(scores_deque) > 30.0):
            print(text + " | noise {}".format(agent.noise.sigma))
            agent.save_checkpoints()
            
        else:
            print(text, end="")
        
        if np.mean(scores_deque) >= 0.5:
            print('Done :)')
            break


In [6]:
scores = []
times = []
train_ddpg(n_episodes = 5000, print_every=100)

Episode 100 | Score Average -0.003, perc 10, 50 , 90: , -0.00, -0.00, -0.00 | Score: -0.00 | Time 14 | noise 0.8594113717731046
Episode 200 | Score Average 0.009, perc 10, 50 , 90: , -0.00, -0.00, 0.05 | Score: 0.05 | Time 33 | noise 0.7047462315860497
Episode 300 | Score Average 0.093, perc 10, 50 , 90: , -0.00, 0.10, 0.20 | Score: 0.05 | Time 32 | noise 0.3976646932421935
Episode 400 | Score Average 0.135, perc 10, 50 , 90: , 0.05, 0.10, 0.30 | Score: 0.05 | Time 32 | noise 0.19491693988844777
Episode 500 | Score Average 0.092, perc 10, 50 , 90: , 0.05, 0.05, 0.20 | Score: 0.05 | Time 45 | noise 0.11197165978971184
Episode 600 | Score Average 0.414, perc 10, 50 , 90: , 0.05, 0.20, 0.90 | Score: 1.90 | Time 760 | noise 0.018642728190668874
Episode 608 | Score Average 0.506, perc 10, 50 , 90: , 0.05, 0.20, 1.60 | Score: 1.60 | Time 642Done :)


In [8]:
data = [go.Scatter(x=np.arange(len(scores)), y=scores)]
layout = dict(
    title = 'Improvement of scores during trainig',
    xaxis = dict(title = 'Number of Episodes'),
    yaxis = dict(title = 'Score'),
)

fig = dict(data=data, layout=layout)
iplot(fig)